In [1]:
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

import acquire
import prepare
import wrangle_excs as wrangle

In [31]:
def wrangle_zillow():
    # To get the single unit properties
    single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]

    # Importing the data into a pandas dataframe
    df = acquire.get_zillow_data()

    # Returns a summary of the dataframe
    acquire.summarize_df(df)

    # Get only single use properties
    df = df[df.propertylandusetypeid.isin(single_use)]

    # Restrict df to only those properties with at least 1 bath & bed and 350 sqft area
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull())\
            & (df.calculatedfinishedsquarefeet>350)]
    
    # removing bedroom counts above 7
    df = df[df.bedroomcnt < 7]
    
    # removing houses over 10000 square feet
    df = df[df.calculatedfinishedsquarefeet < 10000]

    # Removes columns with too high percentage of nulls
    df = acquire.handle_missing_values(df)

    # Get dummie variables for our counties
    df['county'] = df.fips.apply(lambda x: 'orange' if x == 6059.0 else 'los_angeles' if x == 6037.0 else 'ventura')
    df = df.drop(columns=['fips'])
    dummies = pd.get_dummies(df['county'])
    df = pd.concat([df, dummies],axis=1)

    # adding tax rate percentage
    df['taxrate'] = df.taxamount/df.taxvaluedollarcnt*100

    # adding structure square footage cost
    df['structure_dollar_per_sqft'] = df.structuretaxvaluedollarcnt/df.calculatedfinishedsquarefeet

    # adding land square footage cost
    df['land_dollar_per_sqft'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet

    # removing outliers on lotsize
    df = df[df.lotsizesquarefeet < 200000]

    # removing columns that are of no use
    dropcols = ['parcelid',
                'calculatedbathnbr',
                'finishedsquarefeet12',
                'fullbathcnt',
                'heatingorsystemtypeid',
                'propertycountylandusecode',
                'propertylandusetypeid',
                'propertyzoningdesc',
                'censustractandblock',
                'propertylandusedesc',
                'unitcnt',
                'transactiondate',
                'county',
               'heatingorsystemdesc',
               'id']
    
    df.drop(columns = dropcols, inplace = True)

    # filling nulls
    # assume that since this is Southern CA, null means 'None' for heating system
    df.lotsizesquarefeet.fillna(7313, inplace = True)
    df.buildingqualitytypeid.fillna(6.0, inplace = True)

    # Renaming columns for readibility
    df.rename(columns = {"bathroomcnt":"bathrooms",
                        "bedroomcnt":"bedrooms",
                        "buildingqualitytypeid ":"building_quality",
                        "calculatedfinishedsquarefeet":"square_footage",
                        "lotsizesquarefeet":"lot_size",
                        "rawcensustractandblock":"census_tract_and_block",
                        "regionidcity":"city_id",
                        "regionidcounty":"county_id",
                        "regionidzip":"zip_id",
                        "roomcnt":"room_count",
                        "yearbuilt":"year_built",
                        "structuretaxvaluedollarcnt":"structure_tax_value",
                        "taxvaluedollarcnt":"tax_value",
                        "logerror":"target"}, inplace = True)
    # dropping the null values
    df = df.dropna()

    # removing outliers
    df = prepare.remove_outliers(df)

    # train validate test split
    train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.train_validate_test(df, target = "target")

    # scaling our data
    scaler, X_train_scaled, X_validate_scaled, X_test_scaled = prepare.min_max_scaler(X_train, X_validate, X_test)
    
    return df, scaler, train, validate, test, X_train, X_train_scaled, y_train, X_validate, X_validate_scaled, y_validate, X_test, X_test_scaled, y_test

In [32]:
df, scaler, train, validate, test, X_train, X_train_scaled, y_train, X_validate, X_validate_scaled, y_validate, X_test, X_test_scaled, y_test = wrangle_zillow()

-----Head-------
   typeconstructiontypeid  storytypeid  propertylandusetypeid  \
0                     NaN          NaN                  261.0   
1                     NaN          NaN                  261.0   
2                     NaN          NaN                  261.0   

   heatingorsystemtypeid  buildingclasstypeid  architecturalstyletypeid  \
0                    NaN                  NaN                       NaN   
1                    NaN                  NaN                       NaN   
2                    NaN                  NaN                       NaN   

   airconditioningtypeid  parcelid       id  basementsqft  ...  \
0                    NaN  14297519  1727539           NaN  ...   
1                    NaN  17052889  1387261           NaN  ...   
2                    NaN  14186244    11677           NaN  ...   

   censustractandblock  logerror  transactiondate  airconditioningdesc  \
0         6.059063e+13  0.025595       2017-01-01                  NaN   
1       

(31.576, 1286.3]      3145
(1286.3, 2528.6]      2557
(2528.6, 3770.9]       243
(3770.9, 5013.2]        62
(5013.2, 6255.5]        17
(6255.5, 7497.8]         7
(7497.8, 8740.1]         2
(8740.1, 9982.4]         1
(9982.4, 11224.7]        0
(11224.7, 12467.0]       1
Name: finishedsquarefeet50, dtype: int64
(374.781, 901.8]    67
(901.8, 1423.6]     95
(1423.6, 1945.4]    45
(1945.4, 2467.2]    46
(2467.2, 2989.0]    30
(2989.0, 3510.8]    33
(3510.8, 4032.6]    31
(4032.6, 4554.4]    33
(4554.4, 5076.2]     4
(5076.2, 5598.0]     2
Name: finishedsquarefeet6, dtype: int64
(6036.925, 6044.4]    50695
(6044.4, 6051.8]          0
(6051.8, 6059.2]      20630
(6059.2, 6066.6]          0
(6066.6, 6074.0]          0
(6074.0, 6081.4]          0
(6081.4, 6088.8]          0
(6088.8, 6096.2]          0
(6096.2, 6103.6]          0
(6103.6, 6111.0]       6250
Name: fips, dtype: int64
(0.995, 1.4]    7016
(1.4, 1.8]         0
(1.8, 2.2]       998
(2.2, 2.6]         0
(2.6, 3.0]       235
(3.0, 3.4

Name: architecturalstyledesc, dtype: int64
Buildings having wood or wood and steel frames                                                                                                                                                                                             14
Buildings having exterior walls built of a non-combustible material such as brick, concrete, block or poured concrete. Interior partitions and roof structures are built of combustible materials. Floor may be concrete or wood frame.     1
Name: buildingclassdesc, dtype: int64
Central       33633
Floor/Wall    14107
Yes             850
Forced air      776
Solar           104
None             60
Radiant          23
Baseboard        13
Gravity           3
Heat Pump         1
Name: heatingorsystemdesc, dtype: int64
Single Family Residential                     52439
Condominium                                   19341
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       

In [33]:
train.head()

,bathrooms,bedrooms,buildingqualitytypeid,square_footage,latitude,longitude,lot_size,census_tract_and_block,city_id,county_id,...,assessmentyear,landtaxvaluedollarcnt,taxamount,target,los_angeles,orange,ventura,taxrate,structure_dollar_per_sqft,land_dollar_per_sqft
56492,2.0,3.0,6.0,1308.0,33683614.0,-117905359.0,6500.0,6.059064e+07,38032.0,1286.0,...,2016.0,244296.0,4672.77,0.006996,0,1,0,1.470826,56.116972,37.584000
74855,2.0,2.0,8.0,1848.0,34632279.0,-118214952.0,12372.0,6.037910e+07,5534.0,3101.0,...,2016.0,49466.0,2725.68,-0.022607,1,0,0,1.252593,90.983225,3.998222
26254,2.0,3.0,6.0,1490.0,34197656.0,-118519978.0,6689.0,6.037132e+07,12447.0,3101.0,...,2016.0,268544.0,4307.81,0.021316,1,0,0,1.257538,49.674497,40.147107
68956,3.0,2.0,8.0,2044.0,34166600.0,-118522000.0,21799.0,6.037140e+07,12447.0,3101.0,...,2016.0,214406.0,5276.04,0.038740,1,0,0,1.206506,109.047456,9.835589
76854,2.5,2.0,6.0,1250.0,33816269.0,-117976198.0,1550.0,6.059088e+07,16764.0,1286.0,...,2016.0,172732.0,3259.86,-0.001353,0,1,0,1.198689,79.376000,111.440000


In [34]:
df.shape

(58002, 25)

In [24]:
df.head().T

,0,1,2,3,5
id,1.727539e+06,1.387261e+06,1.167700e+04,2.288172e+06,1.447245e+06
bathrooms,3.500000e+00,1.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00
bedrooms,4.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,3.000000e+00
buildingqualitytypeid,6.000000e+00,6.000000e+00,6.000000e+00,8.000000e+00,6.000000e+00
square_footage,3.100000e+03,1.465000e+03,1.243000e+03,2.376000e+03,1.492000e+03
latitude,3.363493e+07,3.444927e+07,3.388617e+07,3.424518e+07,3.423004e+07
longitude,-1.178692e+08,-1.192815e+08,-1.178232e+08,-1.182407e+08,-1.189940e+08
lot_size,4.506000e+03,1.264700e+04,8.432000e+03,1.303800e+04,9.030000e+02
census_tract_and_block,6.059063e+07,6.111001e+07,6.059022e+07,6.037300e+07,6.111005e+07
city_id,5.357100e+04,1.309100e+04,2.141200e+04,3.965510e+05,5.123900e+04


In [9]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'customer_id': 'num_rows'}).reset_index()
    return rows_missing

In [10]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
id,0,0.0
bathrooms,0,0.0
bedrooms,0,0.0
buildingqualitytypeid,0,0.0
square_footage,0,0.0
latitude,0,0.0
longitude,0,0.0
lot_size,0,0.0
census_tract_and_block,0,0.0
city_id,0,0.0


In [11]:
nulls_by_row(df)

,num_cols_missing,percent_cols_missing,index
0,0,0.0,58147


In [12]:
df.head()

,id,bathrooms,bedrooms,buildingqualitytypeid,square_footage,latitude,longitude,lot_size,census_tract_and_block,city_id,...,assessmentyear,landtaxvaluedollarcnt,taxamount,target,los_angeles,orange,ventura,taxrate,structure_dollar_per_sqft,land_dollar_per_sqft
0,1727539,3.5,4.0,6.0,3100.0,33634931.0,-117869207.0,4506.0,6.059063e+07,53571.0,...,2016.0,537569.0,11013.72,0.025595,0,1,0,1.076313,156.681613,119.300710
1,1387261,1.0,2.0,6.0,1465.0,34449266.0,-119281531.0,12647.0,6.111001e+07,13091.0,...,2016.0,376000.0,5672.48,0.055619,0,0,1,1.222517,60.068259,29.730371
2,11677,2.0,3.0,6.0,1243.0,33886168.0,-117823170.0,8432.0,6.059022e+07,21412.0,...,2016.0,479489.0,6488.30,0.005383,0,1,0,1.148823,68.615447,56.865394
3,2288172,3.0,4.0,8.0,2376.0,34245180.0,-118240722.0,13038.0,6.037300e+07,396551.0,...,2016.0,36225.0,1777.51,-0.103410,1,0,0,1.224661,45.840909,2.778417
5,1447245,2.0,3.0,6.0,1492.0,34230044.0,-118993991.0,903.0,6.111005e+07,51239.0,...,2016.0,132424.0,3508.10,-0.020526,0,0,1,1.059644,133.136729,146.648948


In [15]:
df.bedrooms.value_counts()

3.0     24655
4.0     14893
2.0     12448
5.0      3734
1.0      1702
6.0       587
7.0        98
8.0        20
9.0         8
11.0        1
10.0        1
Name: bedrooms, dtype: int64

In [25]:
df.square_footage.max()

12616.0

In [30]:
df.head()

,id,bathrooms,bedrooms,buildingqualitytypeid,square_footage,latitude,longitude,lot_size,census_tract_and_block,city_id,...,assessmentyear,landtaxvaluedollarcnt,taxamount,target,los_angeles,orange,ventura,taxrate,structure_dollar_per_sqft,land_dollar_per_sqft
0,1727539,3.5,4.0,6.0,3100.0,33634931.0,-117869207.0,4506.0,6.059063e+07,53571.0,...,2016.0,537569.0,11013.72,0.025595,0,1,0,1.076313,156.681613,119.300710
1,1387261,1.0,2.0,6.0,1465.0,34449266.0,-119281531.0,12647.0,6.111001e+07,13091.0,...,2016.0,376000.0,5672.48,0.055619,0,0,1,1.222517,60.068259,29.730371
2,11677,2.0,3.0,6.0,1243.0,33886168.0,-117823170.0,8432.0,6.059022e+07,21412.0,...,2016.0,479489.0,6488.30,0.005383,0,1,0,1.148823,68.615447,56.865394
3,2288172,3.0,4.0,8.0,2376.0,34245180.0,-118240722.0,13038.0,6.037300e+07,396551.0,...,2016.0,36225.0,1777.51,-0.103410,1,0,0,1.224661,45.840909,2.778417
5,1447245,2.0,3.0,6.0,1492.0,34230044.0,-118993991.0,903.0,6.111005e+07,51239.0,...,2016.0,132424.0,3508.10,-0.020526,0,0,1,1.059644,133.136729,146.648948


In [36]:
acquire.get_zillow_data()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10833991,2864704,NaN,...,6.037132e+13,-0.002245,2017-09-20,Central,NaN,NaN,Central,Condominium,NaN,NaN
77571,NaN,NaN,261.0,2.0,NaN,NaN,NaN,11000655,673515,NaN,...,6.037101e+13,0.020615,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77572,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17239384,2968375,NaN,...,6.111008e+13,0.013209,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77573,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12773139,1843709,NaN,...,6.037434e+13,0.037129,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
